In [20]:
# !pip uninstall -y pinecone pinecone-client pinecone-text
# !pip install "pinecone-client>=4,<6" "pinecone-text>=0.9.0" "langchain-pinecone>=0.2.0"

In [21]:
import pinecone, inspect, os
print("pinecone module file:", pinecone.__file__)

pinecone module file: /home/dhc99/anaconda3/envs/torch_env/lib/python3.11/site-packages/pinecone/__init__.py


In [22]:
# # 패키지 설치 (필요 시 실행)

# %pip install -U langchain langchain-text-splitters langchain-community langchain-openai langchain-pinecone pinecone langgraph python-dotenv pypdf

# Assignment 04 — 01_nist_rag.ipynb

이 노트북은 NIST PDF 3종을 로드/청킹하여 Pinecone에 업서트하고, LangChain 기반 RAG 체인을 구성하여 한국어 질의를 수행합니다. LangSmith로 모든 실행을 트레이스합니다.

# 1) 환경 설정 및 라이브러리 설치

다음 셀은 필요한 라이브러리를 임포트하고, 설치가 필요한 경우 안내를 제공합니다.

In [23]:
# 설치/임포트 코드
import os
import json
from pathlib import Path

# LangChain core (text splitters 위치 변경 대응)
try:
    from langchain_text_splitters import RecursiveCharacterTextSplitter
except ImportError:
    # 구버전 호환
    from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# PDF loader
from langchain_community.document_loaders import PyPDFLoader

# Embeddings & VectorStore
from langchain_openai import OpenAIEmbeddings
# from langchain_community.vectorstores import Pinecone # Deprecated or causes conflict
from langchain_pinecone import PineconeVectorStore

# LLM
from langchain_openai import ChatOpenAI

# LangSmith / dotenv
from dotenv import load_dotenv
print("Imports OK (RecursiveCharacterTextSplitter loaded)")

Imports OK (RecursiveCharacterTextSplitter loaded)


In [24]:
# 2) LangSmith/키 설정 (.env 로드)
from dotenv import load_dotenv
load_dotenv()

# Required keys
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
# PINECONE_ENVIRONMENT is optional if index already exists

# LangSmith tracing
os.environ["LANGSMITH_TRACING"] = os.getenv("LANGSMITH_TRACING", "true")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY", "")
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT", "nist-rag-project")

assert OPENAI_API_KEY, "OPENAI_API_KEY missing"
assert PINECONE_API_KEY, "PINECONE_API_KEY missing"

print("Env OK: LangSmith + OpenAI + Pinecone")

Env OK: LangSmith + OpenAI + Pinecone


In [25]:
# 3) NIST PDF 로드 (LangChain Loader)
import os
from pathlib import Path

# 현재 작업 디렉토리 확인 (디버깅용)
print(f"Current working directory: {os.getcwd()}")

# 데이터 디렉토리 찾기
# 1. 워크스페이스 루트 기준
DATA_DIR = Path("assignment04/data")
if not DATA_DIR.exists():
    # 2. 노트북 위치 기준 (assignment04 폴더 내부일 경우)
    DATA_DIR = Path("data")

if not DATA_DIR.exists():
    print("Error: Could not find data directory.")
else:
    print(f"Data directory found at: {DATA_DIR.absolute()}")

# 실제 파일명에 맞춰 수정
PDFS = [
    (DATA_DIR / "nist_ai_risk_framework.pdf", "ai_rmf", "AI RMF"),
    (DATA_DIR / "nist_cybersecurity_framework.pdf", "csf_2_0", "NIST CSF 2.0"),
    (DATA_DIR / "nist_zero_trust.pdf", "zero_trust", "Zero Trust"),
]

all_docs = []
for path, source, doc_title in PDFS:
    # 파일 존재 여부 확인
    if not path.exists():
        print(f"Warning: File not found at {path}")
    else:
        try:
            loader = PyPDFLoader(str(path))
            docs = loader.load()
            for d in docs:
                d.metadata = {
                    **d.metadata,
                    "source": source,
                    "doc_title": doc_title,
                    # retain page number from loader metadata if present
                    "page": d.metadata.get("page", d.metadata.get("page_number", None))
                }
            all_docs.extend(docs)
            print(f"Loaded {len(docs)} pages from {path.name}")
        except Exception as e:
            print(f"Error loading {path}: {e}")

print(f"Total loaded pages: {len(all_docs)}")
if all_docs:
    print("Example page metadata:", all_docs[0].metadata)
else:
    print("No pages loaded. Please check the file paths and ensure PyPDFLoader is working.")

Current working directory: /home/dhc99/ajou-llmops-2025-2nd-semester/assignment04
Data directory found at: /home/dhc99/ajou-llmops-2025-2nd-semester/assignment04/data
Loaded 48 pages from nist_ai_risk_framework.pdf
Loaded 48 pages from nist_ai_risk_framework.pdf
Loaded 32 pages from nist_cybersecurity_framework.pdf
Loaded 32 pages from nist_cybersecurity_framework.pdf
Loaded 59 pages from nist_zero_trust.pdf
Total loaded pages: 139
Example page metadata: {'producer': 'pdfTeX-1.40.24', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-01-24T14:45:46-05:00', 'author': 'National Institute of Standards and Technology', 'keywords': 'Artificial Intelligence (AI); AI; AI RMF; AI RMF 1.0; AI systems; trustworthy and responsible AI.', 'moddate': '2025-06-04T13:01:45-04:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.24 (TeX Live 2022) kpathsea version 6.3.4', 'subject': 'As directed by the National Artificial Intelligence Initiative Act of 2020 (P.L. 116-283), the go

In [26]:
# 4) 텍스트 청크 분할 및 메타데이터 부착
if not all_docs:
    print("Warning: No documents to split. Please check the previous cell.")
    chunked_docs = []
else:
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=900,
        chunk_overlap=150,
        separators=["\n\n", "\n", " "]
    )
    chunked_docs = splitter.split_documents(all_docs)
    print(f"Chunks: {len(chunked_docs)}")
    if chunked_docs:
        print(chunked_docs[0].metadata)
    else:
        print("Warning: Documents were split but resulted in 0 chunks.")

Chunks: 484
{'producer': 'pdfTeX-1.40.24', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-01-24T14:45:46-05:00', 'author': 'National Institute of Standards and Technology', 'keywords': 'Artificial Intelligence (AI); AI; AI RMF; AI RMF 1.0; AI systems; trustworthy and responsible AI.', 'moddate': '2025-06-04T13:01:45-04:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.24 (TeX Live 2022) kpathsea version 6.3.4', 'subject': 'As directed by the National Artificial Intelligence Initiative Act of 2020 (P.L. 116-283), the goal of the AI RMF is to offer a resource to the organizations designing, developing, deploying, or using AI systems to help manage the many risks of AI and promote trustworthy and responsible development and use of AI systems. The Framework is intended to be voluntary, rights-preserving, non-sector specific, and use-case agnostic, providing flexibility to organizations of all sizes and in all sectors and throughout society to implement the appr

In [27]:
# 5) 임베딩 선택 및 Pinecone 인덱스 생성
from pinecone import Pinecone, ServerlessSpec
import os

INDEX_NAME = "nist-rag-index"
NAMESPACE = "nist"  # 한 과제 전체에서 공통 사용

# Embeddings
# 사용자가 생성한 인덱스(1024차원)에 맞춰 차원 축소 설정
embeddings = OpenAIEmbeddings(model="text-embedding-3-large", dimensions=1024)

# Init Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

# Create index if not exists
if INDEX_NAME not in [idx.name for idx in pc.list_indexes()]:
    # Default to us-east-1 if not set, though user likely has index already
    env = os.getenv("PINECONE_ENVIRONMENT", "us-east-1")
    pc.create_index(
        name=INDEX_NAME,
        dimension=1024, # 1024차원으로 설정
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region=env)
    )
    print("Created index", INDEX_NAME)
else:
    print("Using existing index", INDEX_NAME)

Using existing index nist-rag-index


In [28]:
# 6) 청크 업서트 (VectorStore)
# pinecone 클라이언트(pc)와 INDEX_NAME, NAMESPACE, embeddings는 이전 셀에서 생성됨

# PineconeVectorStore (langchain-pinecone) 사용
vectorstore = PineconeVectorStore.from_documents(
    documents=chunked_docs,
    embedding=embeddings,
    index_name=INDEX_NAME,
    namespace=NAMESPACE
)
print("Upserted", len(chunked_docs), "chunks to Pinecone (namespace=", NAMESPACE, ")")

Upserted 484 chunks to Pinecone (namespace= nist )


In [32]:
# 7) VectorStore 검색 기본 테스트
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

q1 = "AI RMF에서 위험 평가의 핵심 단계가 뭐야?"
# 최신 LangChain 버전에서는 invoke() 사용 권장
res1 = retriever.invoke(q1)
print(len(res1), res1[0].metadata)

# metadata filter example: only zero_trust
res2 = vectorstore.similarity_search(
    "제로 트러스트 원칙을 요약해줘",
    k=3,
    filter={"source": "zero_trust"}
)
print(len(res2), res2[0].metadata)

4 {'author': 'National Institute of Standards and Technology', 'creationdate': '2023-01-24T14:45:46-05:00', 'creator': 'LaTeX with hyperref', 'doc_title': 'AI RMF', 'keywords': 'Artificial Intelligence (AI); AI; AI RMF; AI RMF 1.0; AI systems; trustworthy and responsible AI.', 'moddate': '2025-06-04T13:01:45-04:00', 'page': 11.0, 'page_label': '7', 'producer': 'pdfTeX-1.40.24', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.24 (TeX Live 2022) kpathsea version 6.3.4', 'source': 'ai_rmf', 'subject': 'As directed by the National Artificial Intelligence Initiative Act of 2020 (P.L. 116-283), the goal of the AI RMF is to offer a resource to the organizations designing, developing, deploying, or using AI systems to help manage the many risks of AI and promote trustworthy and responsible development and use of AI systems. The Framework is intended to be voluntary, rights-preserving, non-sector specific, and use-case agnostic, providing flexibility to organizations of all siz

In [33]:
# 8) RAG Q&A 체인 구성 (retriever → prompt → llm → parser)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 NIST 문서를 기반으로 답변하는 보조자다. 반드시 어떤 문서/페이지를 한 문장으로 먼저 언급하고, 그 다음에 내용을 자세히 설명해라. 모르면 모른다고 답해라."),
    ("human", "질문: {question}\n\n다음은 관련 문서 청크들이다:\n{context}\n\n위 청크들만을 근거로, 한국어로 자세히 답변해라.")
])

# format retrieved docs

def format_docs(docs):
    parts = []
    for d in docs:
        meta = d.metadata
        parts.append(f"[source={meta.get('source')}, doc_title={meta.get('doc_title')}, page={meta.get('page')}]\n{d.page_content}")
    return "\n\n".join(parts)

rag_chain = (
    {"context": retriever | format_docs, "question": lambda x: x}
    | prompt
    | llm
    | StrOutputParser()
)

print("RAG chain ready")

RAG chain ready


In [34]:
# 9) LangSmith 트레이싱 설정과 RAG 질의 5개 실행
questions = [
    "AI RMF의 코어 기능을 간단히 설명해줘",
    "CSF 2.0에서 식별(Identify) 기능의 주요 항목은?",
    "Zero Trust에서 자산 접근 제어의 원칙은 무엇이야?",
    "AI 위험 평가에서 데이터 거버넌스 관련 지침은?",
    "CSF 2.0의 보호(Protect) 영역에서 권장 통제는?"
]

results = []
for q in questions:
    out = rag_chain.with_config({
        "tags": ["nist", "rag"],
        "run_name": "nist_rag_qa"
    }).invoke(q)
    results.append({"question": q, "answer": out})
    print("Q:", q)
    print("A:", out[:500], "...\n")

# save minimal results
out_dir = Path("assignment04/results")
out_dir.mkdir(parents=True, exist_ok=True)
with open(out_dir / "rag_results.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print("Saved results to", out_dir / "rag_results.json")

Q: AI RMF의 코어 기능을 간단히 설명해줘
A: NIST AI RMF 1.0 문서의 24페이지에 따르면, AI RMF의 코어는 AI 위험을 관리하고 신뢰할 수 있는 AI 시스템을 책임감 있게 개발하기 위한 대화, 이해 및 활동을 가능하게 하는 결과와 행동을 제공합니다. 이 코어는 GOVERN, MAP, MEASURE, MANAGE의 네 가지 기능으로 구성되어 있으며, 각 기능은 카테고리와 하위 카테고리로 세분화됩니다. 이러한 카테고리와 하위 카테고리는 특정 행동과 결과로 나뉘어져 있습니다. 

이 네 가지 기능은 AI 위험 관리 활동을 조직하는 데 있어 가장 높은 수준에서 작동하며, GOVERN(거버넌스)는 다른 세 가지 기능에 걸쳐 정보를 제공하고 통합되는 교차 기능으로 설계되었습니다. 사용자는 자신의 자원과 능력에 따라 이러한 기능을 적용할 수 있으며, 필요에 따라 카테고리와 하위 카테고리를 선택하거나 모든 카테고리를 적용할 수 있습니다. 

또한, 이러한 기능은 AI 생애 주기 전반에 걸쳐 사용자가 가치를 추가한다고 판단하는 ...

Q: CSF 2.0에서 식별(Identify) 기능의 주요 항목은?
A: NIST CSF 2.0 문서의 페이지 19.0에 따르면, 식별(Identify) 기능의 주요 항목은 다음과 같습니다: 자산 관리(ID.AM), 위험 평가(ID.RA), 개선(ID.IM)입니다.

식별 기능은 조직의 사이버 보안 환경을 이해하고 관리하기 위한 기초를 제공합니다. 자산 관리(ID.AM)는 조직이 보유한 자산을 식별하고 관리하는 과정을 포함하며, 위험 평가(ID.RA)는 사이버 위험을 평가하여 조직의 보안 태세를 이해하는 데 도움을 줍니다. 마지막으로, 개선(ID.IM)은 사이버 보안 프로세스와 관행을 지속적으로 개선하기 위한 활동을 포함합니다. 이러한 항목들은 조직이 사이버 보안 위험을 효과적으로 관리하고 대응하기 위한 기반을 마련하는 데 필수적입니다. ...

Q: CSF 2.0에서 식별(Identify) 기능의 주요 항목은?
A: 